# <center> <img src="img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ingeniería en Sistemas Computacionales** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 11**: Multi classification (Decision Trees & SVM)

**Fecha**: 29/04/25

**Nombre del Estudiante**: Angel Ramirez, Roberto Osorno, Yochabel Cazares, Samuel Romero

**Profesor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MLSpark-Decision-Trees") \
    .master("spark://f04d2745dc57:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/29 17:27:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Preparación de datos

In [3]:
from team_name.spark_utils import SparkUtils

iris_schema = SparkUtils.generate_schema([
    ("Id", "integer"),
    ("SepalLengthCm", "double"),
    ("SepalWidthCm", "double"),
    ("PetalLengthCm", "double"),
    ("PetalWidthCm", "double"),
    ("Species", "string")
])

iris_df = spark.read \
               .schema(iris_schema) \
               .option("header", "true") \
               .csv("/home/jovyan/notebooks/data/Iris.csv")

iris_df.printSchema()
iris_df.show(5, truncate=False)

root
 |-- Id: integer (nullable = true)
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)



+---+-------------+------------+-------------+------------+-----------+
|Id |SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|Species    |
+---+-------------+------------+-------------+------------+-----------+
|1  |5.1          |3.5         |1.4          |0.2         |Iris-setosa|
|2  |4.9          |3.0         |1.4          |0.2         |Iris-setosa|
|3  |4.7          |3.2         |1.3          |0.2         |Iris-setosa|
|4  |4.6          |3.1         |1.5          |0.2         |Iris-setosa|
|5  |5.0          |3.6         |1.4          |0.2         |Iris-setosa|
+---+-------------+------------+-------------+------------+-----------+
only showing top 5 rows



### Ensamblar las características en una sola columna vectorial

In [4]:
from pyspark.ml.feature import VectorAssembler, StringIndexer

indexer = StringIndexer(inputCol="Species", outputCol="label")
data_indexed = indexer.fit(iris_df).transform(iris_df)

assembler = VectorAssembler(inputCols=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"], outputCol="features")
data_with_features = assembler.transform(data_indexed).select("label", "features")

### Dividir los datos en conjuntos de entrenamiento y prueba: 80 % de datos de entrenamiento y 20 % de datos de prueba.

In [5]:
train_df, test_df = data_with_features.randomSplit([0.8, 0.2], seed=57)

### Show the whole dataset

In [6]:
print("Original Dataset")
data_with_features.show()

#
print("train set")
train_df.show()

Original Dataset


+-----+-----------------+
|label|         features|
+-----+-----------------+
|  0.0|[5.1,3.5,1.4,0.2]|
|  0.0|[4.9,3.0,1.4,0.2]|
|  0.0|[4.7,3.2,1.3,0.2]|
|  0.0|[4.6,3.1,1.5,0.2]|
|  0.0|[5.0,3.6,1.4,0.2]|
|  0.0|[5.4,3.9,1.7,0.4]|
|  0.0|[4.6,3.4,1.4,0.3]|
|  0.0|[5.0,3.4,1.5,0.2]|
|  0.0|[4.4,2.9,1.4,0.2]|
|  0.0|[4.9,3.1,1.5,0.1]|
|  0.0|[5.4,3.7,1.5,0.2]|
|  0.0|[4.8,3.4,1.6,0.2]|
|  0.0|[4.8,3.0,1.4,0.1]|
|  0.0|[4.3,3.0,1.1,0.1]|
|  0.0|[5.8,4.0,1.2,0.2]|
|  0.0|[5.7,4.4,1.5,0.4]|
|  0.0|[5.4,3.9,1.3,0.4]|
|  0.0|[5.1,3.5,1.4,0.3]|
|  0.0|[5.7,3.8,1.7,0.3]|
|  0.0|[5.1,3.8,1.5,0.3]|
+-----+-----------------+
only showing top 20 rows

train set
+-----+-----------------+
|label|         features|
+-----+-----------------+
|  0.0|[4.3,3.0,1.1,0.1]|
|  0.0|[4.4,2.9,1.4,0.2]|
|  0.0|[4.4,3.0,1.3,0.2]|
|  0.0|[4.4,3.2,1.3,0.2]|
|  0.0|[4.5,2.3,1.3,0.3]|
|  0.0|[4.6,3.1,1.5,0.2]|
|  0.0|[4.6,3.2,1.4,0.2]|
|  0.0|[4.6,3.4,1.4,0.3]|
|  0.0|[4.6,3.6,1.0,0.2]|
|  0.0|[4.7,3.2,1.3,0.2]|
| 

### Crear un modelo de árbol de decisiones

In [7]:
from pyspark.ml.classification import DecisionTreeClassifier

#Inicializar y entrenar el modelo de árbol de decisión
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

### Training

In [8]:
dt_model = dt.fit(train_df)

#Resumen del modelo de visualización
print("Decision Tree model summary:{0}".format(dt_model.toDebugString))

Decision Tree model summary:DecisionTreeClassificationModel: uid=DecisionTreeClassifier_e13477c84b1b, depth=5, numNodes=15, numClasses=3, numFeatures=4
  If (feature 2 <= 2.45)
   Predict: 0.0
  Else (feature 2 > 2.45)
   If (feature 2 <= 4.85)
    If (feature 3 <= 1.65)
     Predict: 1.0
    Else (feature 3 > 1.65)
     If (feature 0 <= 5.95)
      Predict: 1.0
     Else (feature 0 > 5.95)
      Predict: 2.0
   Else (feature 2 > 4.85)
    If (feature 3 <= 1.75)
     If (feature 2 <= 4.95)
      Predict: 1.0
     Else (feature 2 > 4.95)
      If (feature 3 <= 1.55)
       Predict: 2.0
      Else (feature 3 > 1.55)
       Predict: 1.0
    Else (feature 3 > 1.75)
     Predict: 2.0



### Predicciones

In [9]:
#Usar el modelo entrenado para hacer predicciones sobre los datos de prueba
predictions = dt_model.transform(test_df)

#Mostrar las predicciones
predictions.select("features", "prediction").show()

+-----------------+----------+
|         features|prediction|
+-----------------+----------+
|[5.0,3.4,1.6,0.4]|       0.0|
|[5.0,3.5,1.3,0.3]|       0.0|
|[5.1,3.7,1.5,0.4]|       0.0|
|[5.2,3.5,1.5,0.2]|       0.0|
|[5.4,3.9,1.7,0.4]|       0.0|
|[5.7,2.6,3.5,1.0]|       1.0|
|[5.8,2.6,4.0,1.2]|       1.0|
|[5.8,2.7,3.9,1.2]|       1.0|
|[6.1,2.8,4.7,1.2]|       1.0|
|[6.1,2.9,4.7,1.4]|       1.0|
|[6.2,2.9,4.3,1.3]|       1.0|
|[6.3,3.3,4.7,1.6]|       1.0|
|[6.7,3.1,4.7,1.5]|       1.0|
|[4.9,2.5,4.5,1.7]|       1.0|
|[6.0,3.0,4.8,1.8]|       2.0|
|[6.1,3.0,4.9,1.8]|       2.0|
|[6.2,3.4,5.4,2.3]|       2.0|
|[6.3,2.5,5.0,1.9]|       2.0|
|[6.3,2.9,5.6,1.8]|       2.0|
|[6.4,3.2,5.3,2.3]|       2.0|
+-----------------+----------+
only showing top 20 rows



### Evaluar el modelo

In [17]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

accuracy_dt = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})
precision_dt = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
recall_dt = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
f1_dt = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})

print(f"Accuracy: {accuracy_dt}")
print(f"Precision: {precision_dt}")
print(f"Recall: {recall_dt}")
print(f"F1 Score: {f1_dt}")

Accuracy: 0.9545454545454546
Precision: 0.9595959595959596
Recall: 0.9545454545454546
F1 Score: 0.9545454545454545


### SVM

In [ ]:
from pyspark.ml.classification import LinearSVC, OneVsRest

#Crear el modelo base de LinearSVC
lsvc = LinearSVC(maxIter=10, regParam=0.01)

#Convertirlo a un clasificador multi-clase usando OneVsRest
ovr = OneVsRest(classifier=lsvc)

#Entrenar el modelo con el dataset de entrenamiento
ovr_model = ovr.fit(data_with_features)

#Predicciones
predictions_svm = ovr_model.transform(test_df)

### Evaluar el modelo

In [15]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_svm = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
accuracy = evaluator_svm.evaluate(predictions, {evaluator.metricName: "accuracy"})
precision = evaluator_svm.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
recall = evaluator_svm.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
f1 = evaluator_svm.evaluate(predictions, {evaluator.metricName: "f1"})
f1_svm = evaluator_svm.evaluate(predictions_svm)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_svm}")

Accuracy: 0.9545454545454545
Precision: 0.9545454545454545
Recall: 0.9545454545454545
F1 Score: 0.8626794258373206


### Comparar los modelos

In [16]:
print(f"Árbol de decisión vs SVM:")
print(f"Árbol de decisión F1 Score: {f1_dt}, SVM F1 Score: {f1_svm}")

Árbol de decisión vs SVM:
Árbol de decisión F1 Score: 0.9545454545454545, SVM F1 Score: 0.8626794258373206


In [ ]:
spark.stop()